In [4]:
from static_checks.Checker import *
from static_checks.MiniInstantiator import *
from common.datatypes import ForecastingQuestion
from forecasters.consistent_forecaster import ConsistentForecaster
from uuid import UUID
import uuid
import datetime

P = ForecastingQuestion(
        id=UUID("a874f3b3-85b0-4383-a009-efda1ff26357"),
        title="Will a member of the Trump family be the Republican nominee for the US Presidency in 2024?",
        body="Resolution Criteria\nResolves Yes if one of these individuals is the Republican nominee for the US Presidency in 2024:\nDonald Trump\nDonald Trump Jr.\nEric Trump\nIvanka Trump\nTiffany Trump\nMelania Trump\nBarron Trump\n",
        resolution_date=datetime.datetime(2024, 12, 31, 0, 0),
        question_type="binary",
        data_source="synthetic_inst",
        created_date=None,
        url=None,
        metadata=None,
        resolution=None,
    )

Q = ForecastingQuestion(
        id=UUID("a874f3b3-85b0-4383-a009-efda1ff26357"),
        title="Will Trump invent AGI in 2024?",
        body="Resolution Criteria\nResolves Yes if Trump is responsible for the creation of Artificial General Intelligence in 2024. Resolves No otherwise.",
        resolution_date=datetime.datetime(2024, 12, 31, 0, 0),
        question_type="binary",
        data_source="synthetic_inst",
        created_date=None,
        url=None,
        metadata=None,
        resolution=None,
    )

R = ForecastingQuestion(
    id=uuid.uuid4(),
    title="Will the Democrats win the 2028 presidential election in the U.S.?",
    body="Resolves Yes if the Democratic candidate wins the 2028 presidential election in the United States, and No otherwise. "
    "If the candidate is not a member of the Democratic party, but the party endorses them, resolves Yes. "
    "If the party ceases to exist and a candidate from another party wins, resolves No. "
    "If the election is not held between October and December 2028, or if there ceases to be an office of "
    "President of the United States, resolves N/A.",
    question_type="binary",
    resolution_date=datetime.datetime(2028, 12, 31),
    data_source="synthetic",
    url=None,
    metadata={"topics": ["Politics", "Elections", "United States"]},
    resolution=None,
)

bq_tuple = {
    "P": P,
    "Q": Q,
}


x= await CondChecker().instantiate(bq_tuple,model="gpt-4o-mini")


print(x)

Base sentences: {'P': ForecastingQuestion(id=UUID('a874f3b3-85b0-4383-a009-efda1ff26357'), title='Will a member of the Trump family be the Republican nominee for the US Presidency in 2024?', body='Resolution Criteria\nResolves Yes if one of these individuals is the Republican nominee for the US Presidency in 2024:\nDonald Trump\nDonald Trump Jr.\nEric Trump\nIvanka Trump\nTiffany Trump\nMelania Trump\nBarron Trump\n', resolution_date=datetime.datetime(2024, 12, 31, 0, 0), question_type='binary', data_source='synthetic_inst', created_date=None, url=None, metadata=None, resolution=None)}
Instantiating with instantiator Trivial
Instantiation attempt 0
Verifying output: P=ForecastingQuestion(id=UUID('4efc6a12-60b8-4f03-a885-6d01aa71b3ec'), title='Will a member of the Trump family be the Republican nominee for the US Presidency in 2024?', body='Resolution Criteria\nResolves Yes if one of these individuals is the Republican nominee for the US Presidency in 2024:\nDonald Trump\nDonald Trump J

In [3]:
# %%

from static_checks.Checker import *
from time import time

checkers = {
    "para": {"answers": {"P": 0.4, "para_P": 0.8}, "checker": ParaphraseChecker()},
    "neg": {"answers": {"P": 0.6, "not_P": 0.8}, "checker": NegChecker()},
    "cond": {
        "answers": {"P": 0.15, "Q_given_P": 0.6, "P_and_Q": 0.15},
        "checker": CondChecker(),
    },
    "cond2": {
        "answers": {"P": 0.2, "Q_given_P": 1e-6, "P_and_Q": 0.1},
        "checker": CondChecker(),
    },
    "andor": {
        "answers": {"P": 0.6, "Q": 0.6, "P_and_Q": 0.55, "P_or_Q": 0.6},
        "checker": AndOrChecker(),
    },
    "and": {
        "answers": {"P": 0.15, "Q": 0.1, "P_and_Q": 0.1},
        "checker": AndChecker(),
    },
    "or": {
        "answers": {"P": 0.2, "Q": 0.3, "P_or_Q": 0.3},
        "checker": OrChecker(),
    },
    "but": {
        "answers": {"P": 0.6, "Q_and_not_P": 0.3, "P_or_Q": 0.94},
        "checker": ButChecker(),
    },
    "condcond": {
        "answers": {
            "P": 0.5,
            "Q_given_P": 0.5,
            "R_given_P_and_Q": 0.5,
            "P_and_Q_and_R": 0.16,
        },
        "checker": CondCondChecker(),
    },
    "cons": {"answers": {"P": 0.5, "cons_P": 0.45}, "checker": ConsequenceChecker()},
    "ee": {
        "answers": {
            "P": 0.5,
            "P_given_Q": 0.25,
            "P_given_not_Q": 0.4,
            "Q": 0.15,
        },
        "checker": ExpectedEvidenceChecker(),
    },
    "ee2": {
        "answers": {
            "P": 0.3,
            "P_given_Q": 0.35,
            "P_given_not_Q": 0.4,
            "Q": 0.45,
        },
        "checker": ExpectedEvidenceChecker(),
    },
    "ee3": {
        "answers": {
            "P": 0.3,
            "P_given_Q": 0.2,
            "P_given_not_Q": 0.2,
            "Q": 0.65,
        },
        "checker": ExpectedEvidenceChecker(),
    },
    "ee4": {
        "answers": {
            "P": 0.3,
            "P_given_Q": 0.0,
            "P_given_not_Q": 0.3,
            "Q": 0.1,
        },
        "checker": ExpectedEvidenceChecker(),
    },
    "ee5": {
        "answers": {
            "P": 0.25,
            "P_given_Q": 0.0,
            "P_given_not_Q": 0.35,
            "Q": 0.75,
        },
        "checker": ExpectedEvidenceChecker(),
    },    
}

for k, v in checkers.items():
    checker = v["checker"]
    # if k not in ["and", "or"]:
    #     continue
    answers = v["answers"]
    print("Checker:", k)
    time0 = time()
    result_plain = checker.max_min_arbitrage(answers)
    time1 = time()
    print("No kwargs:", result_plain, "\nTime:", time1 - time0)
    # result_weighted = checker.max_min_arbitrage(answers, scoring=[0.5, 0.2])
    # print("Weighted:", result_weighted, "\nTime:", time() - time1)
    try:
        result_de = checker.de_method(answers, tmax=10, euler=False)
    except Exception as e:
        result_de = None
    time2 = time()
    print("DE:", result_de, "\nTime:", time2 - time1)
    try:
        result_de = checker.de_method(answers, tmax=1, euler=False)
    except Exception as e:
        result_de = None
    time3 = time()
    print("DE:", result_de, "\nTime:", time3 - time2)
    result_shgo = checker.max_min_arbitrage(answers, methods=("shgo",))
    time4 = time()
    print("SHGO:", result_shgo, "\nTime:", time4 - time3)
    print("\n")



Checker: para
No kwargs: ({'P': 0.6202041028867288, 'para_P': 0.6202041028867288}, 0.18402096936348789) 
Time: 0.0002002716064453125
DE: ({'P': 0.6200992043715761, 'para_P': 0.6200992043715757}, 0.18368266980469716) 
Time: 0.0422825813293457
DE: ({'P': 0.6200992043715761, 'para_P': 0.6200992043715757}, 0.18368266980469716) 
Time: 0.029800891876220703
SHGO: ({'P': 0.6202041028697819, 'para_P': 0.6202040977296525}, 0.18402096102103532) 
Time: 0.07105422019958496


Checker: neg
No kwargs: ({'P': 0.37979589711327116, 'not_P': 0.6202041028867289}, 0.18402096936348733) 
Time: 7.367134094238281e-05
DE: ({'P': 0.38174973777498, 'not_P': 0.61825026222502}, 0.17771038630846614) 
Time: 0.0032341480255126953
DE: ({'P': 0.38174973777498, 'not_P': 0.61825026222502}, 0.17771038630846614) 
Time: 0.01935267448425293
SHGO: ({'P': 0.3797958855199512, 'not_P': 0.6202040889991406}, 0.18402096566427245) 
Time: 0.059607744216918945


Checker: cond
No kwargs: ({'P': 0.17493381836940514, 'Q_given_P': 0.6670742

/home/abhim/consistency-forecasting/src/static_checks/Checker.py:863: RuntimeWarning: invalid value encountered in log
  scoring_functions[key] = lambda x, sf=scoring_item: sf * np.log(


DE: ({'P': 0.1999995452878426, 'Q_given_P': -1.000000003192566e-06, 'P_and_Q': 0.0003300952099220525}, nan) 
Time: 0.10016870498657227
SHGO: ({'P': 0.20015996529859034, 'Q_given_P': 0.0010000000000005159, 'P_and_Q': 0.0010000000000008411}, 0.10416002179335333) 
Time: 0.032999277114868164


Checker: andor
No kwargs: ({'P': 0.5801334321242474, 'Q': 0.5801334300957942, 'P_and_Q': 0.570236125559228, 'P_or_Q': 0.619543016550925}, 0.0008416841712273748) 
Time: 0.07778787612915039
DE: None 
Time: 0.0004189014434814453
DE: None 
Time: 0.0003237724304199219
SHGO: ({'P': 0.5801334321242474, 'Q': 0.5801334300957942, 'P_and_Q': 0.570236125559228, 'P_or_Q': 0.619543016550925}, 0.0008416841712273748) 
Time: 0.07444000244140625


Checker: and
No kwargs: ({'P': 0.15000000002049665, 'Q': 0.10000031714425833, 'P_and_Q': 0.10000281573708156}, -3.481008336359004e-06) 
Time: 0.06547260284423828
DE: None 
Time: 0.0003802776336669922
DE: None 
Time: 0.00018358230590820312
SHGO: ({'P': 0.15000000002049665, 'Q

/home/abhim/consistency-forecasting/venv/lib/python3.11/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


SHGO: ({'P': 0.4363630496557125, 'P_given_Q': 0.3009706035555113, 'P_given_not_Q': 0.4626877323758402, 'Q': 0.1449752718968944}, 0.015346055778096446) 
Time: 0.140364408493042


Checker: ee2
No kwargs: ({'P': 0.3351184064270629, 'Q': 0.4522564405483057, 'P_given_Q': 0.3087871897616647, 'P_given_not_Q': 0.35685935930395907}, -0.009577591851671396) 
Time: 0.12717103958129883
DE: ({'P': 0.3351184064270629, 'Q': 0.4522564405483057, 'P_given_Q': 0.3087871897616647, 'P_given_not_Q': 0.35685935930395907}, -0.009577591851671396) 
Time: 0.1385488510131836
DE: ({'P': 0.3351184064270629, 'Q': 0.4522564405483057, 'P_given_Q': 0.3087871897616647, 'P_given_not_Q': 0.35685935930395907}, -0.009577591851671396) 
Time: 0.1367185115814209
SHGO: ({'P': 0.3310124844895725, 'P_given_Q': 0.3180543403565667, 'P_given_not_Q': 0.366060679469377, 'Q': 0.45174437002017753}, 0.006531558972059037) 
Time: 0.1131281852722168


Checker: ee3
No kwargs: ({'P': 0.24355289274324537, 'Q': 0.65, 'P_given_Q': 0.2435528927432

/home/abhim/consistency-forecasting/src/static_checks/Checker.py:863: RuntimeWarning: divide by zero encountered in log
  scoring_functions[key] = lambda x, sf=scoring_item: sf * np.log(


No kwargs: ({'P': 0.26713942755601844, 'Q': 0.101027444998519, 'P_given_Q': -1.7726773201942803e-07, 'P_given_not_Q': 0.2971611103104381}, nan) 
Time: 0.08114218711853027
DE: ({'P': 0.26713942755601844, 'Q': 0.101027444998519, 'P_given_Q': -1.7726773201942803e-07, 'P_given_not_Q': 0.2971611103104381}, nan) 
Time: 0.08122849464416504
DE: ({'P': 0.26713942755601844, 'Q': 0.101027444998519, 'P_given_Q': -1.7726773201942803e-07, 'P_given_not_Q': 0.2971611103104381}, nan) 
Time: 0.07748079299926758
SHGO: ({'P': 0.2999571487116477, 'P_given_Q': 0.4978396795734839, 'P_given_not_Q': 0.30004285471412806, 'Q': 0.18117645647828232}, -0.09452616524509985) 
Time: 0.08911991119384766


Checker: ee5
No kwargs: ({'P': 0.15173508413222658, 'Q': 0.6971612915640066, 'P_given_Q': 8.666907316746108e-07, 'P_given_not_Q': 0.5010399202995145}, 0.05006250030008376) 
Time: 0.03519725799560547
DE: ({'P': 0.15173508413222658, 'Q': 0.6971612915640066, 'P_given_Q': 8.666907316746108e-07, 'P_given_not_Q': 0.50103992

In [ ]:
# %%

from forecasters.consistent_forecaster import ConsistentForecaster
from forecasters.basic_forecaster import BasicForecaster
from common.datatypes import ForecastingQuestion
from static_checks.Checker import *
from time import time

cf = ConsistentForecaster(
    BasicForecaster(),
    checks=[
        NegChecker(),
        AndOrChecker(),
        CondChecker(),
        ParaphraseChecker(),
    ],
)
fq = ForecastingQuestion(
    title="Will Manhattan have a skyscraper a mile tall by 2030?",
    body=(
        "Resolves YES if at any point before 2030, there is at least "
        "one building in the NYC Borough of Manhattan (based on current "
        "geographic boundaries) that is at least a mile tall."
    ),
    resolution_date="2030-01-01T00:00:00",
    question_type="binary",
    data_source="manifold",
    url="https://www.metaculus.com/questions/12345/",
    metadata={"foo": "bar"},
    resolution=None,
)

import asyncio

# x = cf.call(fq)
x = asyncio.run(cf.call_async(
    fq,
    bq_func_kwargs={"model": "gpt-4o-mini"},
    instantiation_kwargs={"model": "gpt-4o-mini"},
    model="gpt-4o-mini",
))
# x = asyncio.run(cf.call_async(fq))

In [ ]:
# %%

print(f"{x=}")

raise

In [ ]:
# %%

from static_checks.Checker import *
from time import time

checkers = {
    "para": {"answers": {"P": 0.4, "para_P": 0.43}, "checker": ParaphraseChecker()},
    "neg": {"answers": {"P": 0.6, "not_P": 0.57}, "checker": NegChecker()},
    "cond": {
        "answers": {"P": 0.15, "Q_given_P": 0.6, "P_and_Q": 0.15},
        "checker": CondChecker(),
    },
    "andor": {
        "answers": {"P": 0.6, "Q": 0.6, "P_and_Q": 0.55, "P_or_Q": 0.6},
        "checker": AndOrChecker(),
    },
    "but": {
        "answers": {"P": 0.6, "Q_and_not_P": 0.3, "P_or_Q": 0.94},
        "checker": ButChecker(),
    },
    "condcond": {
        "answers": {
            "P": 0.5,
            "Q_given_P": 0.5,
            "R_given_P_and_Q": 0.5,
            "P_and_Q_and_R": 0.16,
        },
        "checker": CondCondChecker(),
    },
    "cons": {"answers": {"P": 0.5, "cons_P": 0.45}, "checker": ConsequenceChecker()},
}

for k, v in checkers.items():
    checker = v["checker"]
    answers = v["answers"]
    if k != "cons":
        continue
    print("Checker:", k)
    time0 = time()
    result_shgo = checker.max_min_arbitrage(answers, methods=("shgo",))
    time1 = time()
    print("SHGO:", result_shgo, "\nTime:", time1 - time0)
    result_diff_evol = checker.max_min_arbitrage(
        answers, methods=("differential_evolution",)
    )
    time2 = time()
    print("Diff Evolution:", result_diff_evol, "\nTime:", time2 - time1)
    result_dual_annealing = checker.max_min_arbitrage(
        answers, methods=("dual_annealing",)
    )
    time3 = time()
    print("Dual Annealing:", result_dual_annealing, "\nTime:", time3 - time2)
    result_basinhopping = checker.max_min_arbitrage(answers, methods=("basinhopping",))
    time4 = time()
    print("Basinhopping:", result_basinhopping, "\nTime:", time4 - time3)
    result_plain = checker.max_min_arbitrage(answers)
    time5 = time()
    print("No kwargs:", result_plain, "\nTime:", time5 - time4)
    print("\n\n")


# print('----------------')
# print(CondChecker().min_arbitrage(ex, exa))
# print('----------------')
# print(CondChecker().arbitrage(outcome1, ex, exa))
# print(CondChecker().arbitrage(outcome2, ex, exa))
# print(CondChecker().arbitrage(outcome3, ex, exa))
# print('----------------')

In [ ]:
# %%

import json
from common.datatypes import (
    ForecastingQuestion_stripped,
    ForecastingQuestion,
    Prob_cot,
    Prob,
    PlainText,
)
from common.llm_utils import query_api_chat_sync, query_api_chat_sync_native
import os

fq = ForecastingQuestion(
    title="Will Manhattan have a skyscraper a mile tall by 2030?",
    body=(
        "Resolves YES if at any point before 2030, there is at least "
        "one building in the NYC Borough of Manhattan (based on current "
        "geographic boundaries) that is at least a mile tall."
    ),
    resolution_date="2030-01-01T00:00:00",
    question_type="binary",
    data_source="manifold",
    url="https://www.metaculus.com/questions/12345/",
    metadata={"foo": "bar"},
    resolution=None,
)

fqs = ForecastingQuestion_stripped(
    title="Will Manhattan have a skyscraper a mile tall by 2030?",
    body=(
        "Resolves YES if at any point before 2030, there is at least "
        "one building in the NYC Borough of Manhattan (based on current "
        "geographic boundaries) that is at least a mile tall."
    ),
)

print(fqs.__str__())

In [ ]:
# %%

# os.environ["USE_OPENROUTER"] = "True"
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant. Summarize the question for the user.",
    },
    {"role": "user", "content": fq.__str__()},
]
# response = query_api_chat_sync(messages=messages, verbose=True, model="mistralai/mistral-large")